In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Implementación del método de Numerov - Pozo de potencial infinito unidimensional

El método de Numerov es una técnica numérica eficiente para resolver ecuaciones diferenciales de la forma:

$$ \frac{d^2 \psi}{dx^2} = k(x) \psi $$

donde \( k(x) \) depende de la energía y el potencial.

In [8]:

def numerov_step(psi_n, psi_nm1, k_n, k_nm1, k_np1, dx):
    """Un paso del método de Numerov"""
    factor = (2 * (1 - (5/12) * dx**2 * k_n) * psi_n - (1 + (1/12) * dx**2 * k_nm1) * psi_nm1) \
             / (1 + (1/12) * dx**2 * k_np1)
    return factor

### Función `numerov_solve`

Esta función resuelve la ecuación de Schrödinger usando el método de Numerov.

In [2]:
def numerov_solve(E, x, dx, m, hbar):
    """Resuelve la ecuación de Schrödinger con el método de Numerov para energía E"""
    N = len(x)
    psi = np.zeros(N)

    # Definimos k^2 para el pozo infinito (V = 0 dentro del pozo)
    k2 = (2 * m * E / hbar**2) * np.ones(N)

    # Condiciones iniciales
    psi[0] = 0  # Condición de frontera en x=0
    psi[1] = dx  # Pequeño valor arbitrario para iniciar

    for i in range(1, N-1):
        psi[i+1] = numerov_step(psi[i], psi[i-1], k2[i], k2[i-1], k2[i+1], dx)

    return psi

### Definición de parámetros físicos

In [3]:
# Parámetros del problema
L = 1e-9  # Longitud del pozo en metros (1 nm)
N = 1000  # Número de puntos
x = np.linspace(0, L, N)
dx = x[1] - x[0]

# Constantes físicas
m_e = 9.109e-31  # Masa del electrón en kg
hbar = 1.055e-34  # Constante de Planck reducida en J·s

### Función para graficar la función de onda

In [4]:
def plot_wavefunction(n):
    """Grafica la función de onda para el nivel de energía n con el pozo de potencial"""
    E_n = (n**2 * np.pi**2 * hbar**2) / (2 * m_e * L**2)
    psi = numerov_solve(E_n, x, dx, m_e, hbar)
    psi /= np.max(np.abs(psi))  # Normalizar

    plt.figure(figsize=(8, 5))
    plt.plot(x, psi, label=f'n = {n}, E = {E_n:.2e} J')

   # Dibujar el pozo de potencial como una línea roja gruesa en los bordes
    plt.axvline(0, color='red', linestyle='--', linewidth=2, label='Pared del pozo')
    plt.axvline(L, color='red', linestyle='--', linewidth=2)

    # Línea horizontal en y = 0 dentro del rango [0, L]
    plt.plot([0, L], [0, 0], color='red', linestyle='--', linewidth=2)

    plt.xlabel('x (m)')
    plt.ylabel('ψ(x)')
    plt.title('Solución de la ecuación de Schrödinger con Numerov para un electrón')
    plt.legend()
    plt.grid()
    plt.show()


### Interfaz interactiva con widgets

In [9]:
# Crear widget para seleccionar el nivel de energía
n_slider = widgets.IntSlider(min=1, max=10, step=1, value=1, description='Nivel n:')
widgets.interactive(plot_wavefunction, n=n_slider)

interactive(children=(IntSlider(value=1, description='Nivel n:', max=10, min=1), Output()), _dom_classes=('wid…